## Example of converting maps of https://movingai.com/benchmarks/ to YAML one

In [1]:
import re
import os
import yaml

In [2]:
def create_yaml_map_file(map_file, output_file, x_start, y_start, grid_size=50):
    """grid map -> yaml map"""
    r_map = re.compile(r"map")
    r_width = re.compile(r"width\s(\d+)")
    r_height = re.compile(r"height\s(\d+)")
    width = 0
    height = 0

    y = 0
    yaml_dict = {}
    node_exists = []
    
    flg_map = False
    with open(map_file, "r") as f:
        for row in f:
            if flg_map:
                for x, s in enumerate(row.strip()):
                    node_exists.append(s == '.')
                    if s != '.': continue
                    id = str(y * width + x)
                    yaml_dict[id] = {}
                    yaml_dict[id]['pos'] = {
                        'x': x * grid_size + x_start,
                        'y': y * grid_size + y_start
                    }
                    yaml_dict[id]['neigh'] = []
                y += 1
                continue
            m = re.match(r_width, row)
            if m:
                width = int(m.group(1))
                continue
            m = re.match(r_height, row)
            if m:
                height = int(m.group(1))
                continue
            flg_map = bool(re.match(r_map, row))
    
    # create neighbors
    for node_id in yaml_dict.keys():
        id = int(node_id)
        x = id % width
        y = id // width
        # left
        if x > 0 and node_exists[y*width+x-1]:
            yaml_dict[node_id]['neigh'].append(str(y*width+x-1))
        # right
        if x < width - 1 and node_exists[y*width+x+1]:
            yaml_dict[node_id]['neigh'].append(str(y*width+x+1))
        # up
        if y > 0 and node_exists[(y-1)*width+x]:
            yaml_dict[node_id]['neigh'].append(str((y-1)*width+x))
        # down
        if y < height - 1 and node_exists[(y+1)*width+x]:
            yaml_dict[node_id]['neigh'].append(str((y+1)*width+x))

    with open(output_file, "w") as f:
        yaml.dump(yaml_dict, f)

In [ ]:
map_file = "xxxxxxxx.map"
output_file = map_file.split("/")[-1].replace(".map", ".yaml")
create_yaml_map_file(map_file, output_file, x_start=660, y_start=480, grid_size=67)